LM75B 
This is a test to communicate with the temperature circuit LM75B.


In [1]:
"""
   DWF Python Example
   Author:  Digilent, Inc.
   Revision:  2018-07-23

   Requires:                       
       Python 2.7, 3
"""

from ctypes import *
import math
import sys
import time

if sys.platform.startswith("win"):
    dwf = cdll.LoadLibrary("dwf.dll")
elif sys.platform.startswith("darwin"):
    dwf = cdll.LoadLibrary("/Library/Frameworks/dwf.framework/dwf")
else:
    dwf = cdll.LoadLibrary("libdwf.so")

hdwf = c_int()

print("Opening first device")
#dwf.FDwfDeviceOpen(c_int(-1), byref(hdwf))
# device configuration of index 3 (4th) for Analog Discovery has 16kS digital-in/out buffer
dwf.FDwfDeviceConfigOpen(c_int(-1), c_int(3), byref(hdwf)) 

if hdwf.value == 0:
    print("failed to open device")
    szerr = create_string_buffer(512)
    dwf.FDwfGetLastErrorMsg(szerr)
    print(str(szerr.value))
    quit()

#------------- Switch on power to the board - 3.3VDC --------------------

print ("Powering up....")
# enable positive supply
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(0), c_double(True)) 
# set voltage to 3.3 V
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(1), c_double(3.3)) 
# master enable
dwf.FDwfAnalogIOEnableSet(hdwf, c_int(True))

time.sleep (1)    # wait 1 sec to power up

# ------------ Setup I2C communication --------------
print("Configuring I2C...")
iNak = c_int()

dwf.FDwfDigitalI2cRateSet(hdwf, c_double(1e5)) # 100kHz
dwf.FDwfDigitalI2cSclSet(hdwf, c_int(8)) # SCL = DIO-8
dwf.FDwfDigitalI2cSdaSet(hdwf, c_int(9)) # SDA = DIO-9
dwf.FDwfDigitalI2cClear(hdwf, byref(iNak))
if iNak.value == 0:
    print("I2C bus error. Check the pull-ups.")
    quit()
time.sleep(1)


# ------------- Communicate using I2C --------------
rgTX = (c_ubyte*8)(0)
rgRX = (c_ubyte*16)()

#                                8bit address  
dwf.FDwfDigitalI2cWrite(hdwf, c_int(0x48<<1), rgTX, c_int(16), byref(iNak)) # write 16 bytes
time.sleep(0.1)

dwf.FDwfDigitalI2cRead(hdwf, c_int(0x48<<1), rgRX, c_int(16), byref(iNak)) # read 16 bytes
if iNak.value != 0:
    print("NAK "+str(iNak.value))
print(list(rgRX))
time.sleep(0.1)

dwf.FDwfDigitalI2cWriteRead(hdwf, c_int(0x48<<1), rgTX, c_int(1), rgRX, c_int(16), byref(iNak)) # write 1 byte restart and read 16 bytes
if iNak.value != 0:
    print("NAK "+str(iNak.value))
print(list(rgRX))



Opening first device
failed to open device
b'Devices are busy, used by other applications\n'
Powering up....
Configuring I2C...
I2C bus error. Check the pull-ups.
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


Power off and Close handles

In [1]:
#----------- Power off -----------------

# disable positive supply
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(0), c_double(False)) 

# master disable
dwf.FDwfAnalogIOEnableSet(hdwf, c_int(False))


# ---------- Close handle ---------------
dwf.FDwfDeviceCloseAll()

NameError: name 'dwf' is not defined

In [1]:

# ---------- Close handle ---------------
dwf.FDwfDeviceCloseAll()

NameError: name 'dwf' is not defined